In [13]:
from IPython.display import Markdown as md
from pathlib import Path
md(Path("readme.md").read_text())

This repository contains Python code and a Jupyter Notebook
running the [CONTIN program by S. Provencher](http://dx.doi.org/10.1016/0010-4655(82)90174-6)
on every DLS measurement (`*.ASC` files)
at the specified angle found in the given subfolders.


## TODO
- fix multicore (parallel) calc on Windows
- [done] plot measured and fitted correlation curve
- [done] reviewed units of *ptRange* and *fitRange* CONTIN parameters
  - *fitRange* is given in meters now
- [done] output peak statistics with uncertainties
  - by calculating the statistics of lower and upper distribution (uncertainty subtracted from and added to distribution result) and using the max. absolute value
- [done] float formatting in CONTIN input file fixed
- [done] output CONTIN error message if no output was generated

# Some Parameters (please adjust)

## Specify the measurement folder
(And mind the scattering angle in a cell further down!)

In [ ]:
dataDir = '142 2020 MW002-02'
dataDir = '../20210511/094 2021 PS-Standard 1zu1000'

## CONTIN parameters

In [ ]:
continConfig = dict(recalc=False,
    ptRangeSec=(3e-7, 1e0), fitRangeM=(1e-9, 200e-9), gridpts=200,
    transformData=True, freeBaseline=True, weighResiduals=False,
)

In [ ]:
from jupyter_analysis_tools.utils import updatedDict
#angles = [26, 34, 42, 50, 58, 66, 74, 82, 90, 98, 106, 114, 122, 130, 138, 146]
continConfigs = [updatedDict(continConfig, 'angle', angle)
                 for angle in (74, 90,)]
#continConfigs

# Process given data directory

## Find data files

In [ ]:
import jupyter_analysis_tools
jupyter_analysis_tools.utils.setPackage(globals())

In [ ]:
from jupyter_analysis_tools.datalocations import getDataDirs, getDataFiles
dataDirs = getDataDirs(dataDir, noWorkDir=True)
#dataFiles = getDataFiles(dataDirs, include="*raged.ASC")#, exclude="_average")
dataFiles = getDataFiles(dataDirs, include="*.ASC", exclude="_average")
dataFiles

## Run CONTIN on each file

In [ ]:
from .contin import runContinOverFiles

resultDirs = runContinOverFiles(dataFiles, continConfigs, nthreads=None)
#resultDirs

## Fetch CONTIN results

### A single result curve

In [ ]:
from .contin import getContinResults
# show first result for testing
dn = resultDirs[2]
dfDistrib, dfFit, varmap = getContinResults(dn)
dfDistrib.plot('radius', 'distrib', yerr='err', ecolor='salmon', grid=True, label=dn.name);
print(varmap);

### Testing a measure of uncertainties level along the curve

In [ ]:
from jupyter_analysis_tools.distrib import normalizeDistrib, findPeakRanges, findLocalMinima
ranges = findPeakRanges(dfDistrib.radius, dfDistrib.distrib, tol=1e-6)
#findLocalMinima(ranges, dfDistrib.radius.values, dfDistrib.distrib.values, doPlot=True, verbose=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for istart, iend in ranges:
    x = dfDistrib.radius.values[istart:iend+1]
    y = dfDistrib.distrib.values[istart:iend+1]
    u = dfDistrib.err.values[istart:iend+1]
    idx = np.where(y>0)
    div = y[idx]/u[idx]
    plt.errorbar(x, y, yerr=u, ecolor='salmon', label=r"$1/median(y/u)=$"+f"{1/np.median(div):.4g}")
    plt.grid();plt.legend()
    #print(y[idx])
    #print(u[idx])
    print()

## Plot results from all files
(The generalized object-oriented way, for an option of incorporating other methods than CONTIN too)

In [ ]:
import matplotlib.pyplot as plt
from jupyter_analysis_tools.plotting import plotColor, initFigure, lineWidth, GenericResult
from jupyter_analysis_tools.distrib import distrParFromPeakRanges, area, integrate, distrParLatex
from jupyter_analysis_tools.distrib import normalizeDistrib, findPeakRanges
import numpy as np
from .contin import getContinResults
from .dlshelpers import getDLSFileData

class ContinResult(GenericResult):
    name = "CONTIN"
    xColumn = "radius"; yColumn = "distrib"; uColumn = "err"
    getResults = getContinResults
    color = plotColor(1)

class Results:
    def __init__(self, filename, rtype=None):
        self.rtype = rtype
        self.sampleDir = Path(filename)
        self.angle     = None
        self.dfDistrib, self.dfFit, self.varmap = self.rtype.getResults(self.sampleDir, self.angle)
        if self.dfDistrib is None: return
        self.x, self.y, self.u = normalizeDistrib(
            self.dfDistrib[self.rtype.xColumn].values,
            self.dfDistrib[self.rtype.yColumn].values,
            self.dfDistrib[self.rtype.uColumn].values)
        self.peakRanges = findPeakRanges(self.x, self.y, tol=1e-6)
        # refine the peak ranges containing multiple maxima
        self.peakRanges = findLocalMinima(self.peakRanges, self.x, self.y)
        # For a given list of peaks (by start/end indices) return only those
        # whose ratio of amount to uncertainty ratio is always below the given max. ratio
        #maxRatio = 1.5
        #self.peakRanges = [(istart, iend) for istart, iend in self.peakRanges
        #                    if maxRatio > 1/np.median(self.y[istart:iend+1]/self.u[istart:iend+1])]
        # Sort the peaks by area and use the largest (last) only, assuming monomodal distributions
        def peakRangeArea(pr):
            return integrate(self.x[pr[0]:pr[1]+1], self.y[pr[0]:pr[1]+1])
        self.peakRanges = sorted(self.peakRanges, key=peakRangeArea)[-1:]

    def plot(self, axes, subplotIdx=0):
        if self.dfDistrib is None: return
        distPar, _ = distrParFromPeakRanges(self.x, self.y, self.u, self.peakRanges,
                                            plot={'func': self.rtype.plotPeakRange,
                                                  'axes': axes, 'startIdx': subplotIdx+3})
        self.plotCountRate(axes[subplotIdx])
        self.plotCorrelationWithFit(axes[subplotIdx+1])
        self.plotDistibPeaks(axes[subplotIdx+2], distPar)

    def plotCountRate(self, ax):
        indata = getDLSFileData(self.varmap['dataFilename'])
        angle = self.varmap['angle']
        cr = indata['countrate']
        cr.plot(y=angle, ax=ax, grid=True, lw=0.5,
                label=f"@{angle:.0f}°, "+r"$\overline{CR}$="+f"{cr[angle].mean():.1f} kHz",
                xlabel="time (s)", ylabel="Count Rate (kHz)")

    def plotCorrelationWithFit(self, ax):
        """plot fitted correlation curve with residual"""
        ax.plot(self.dfFit['tau'], self.dfFit['corrIn'],
               color="black", lw=lineWidth()*2, label="measured")
        ax.plot(self.dfFit['tau'], self.dfFit['corrFit'],
               color=self.rtype.color, label="fit")
        ax.legend()
        ax2 = ax.twinx()
        residual = self.dfFit['corrIn']-self.dfFit['corrFit']
        ax2.plot(self.dfFit['tau'], residual, 'k.', alpha=.3, label="residual")
        ax2.set_ylim([-max(abs(residual)),max(abs(residual))])
        # combine legends
        ax2handles, ax2labels = ax2.get_legend_handles_labels()
        axhandles, axlabels = ax.get_legend_handles_labels()
        ax.legend(axhandles+ax2handles, axlabels+ax2labels)
        ax.grid(); ax.set_xscale("log");

    def plotDistibPeaks(self, ax, distPar):
        """plot complete distribution as loaded from file"""
        lbl = ("from file, " + self.rtype.name
               + area(self.x, self.y, showArea=True)
               +"\n"+distrParLatex(distPar[0]))
        ax.fill_between(self.x, self.y,
               #width=GenericResult.getBarWidth(self.x),
               color=self.rtype.color, alpha=0.5, label=lbl)
        #ax.errorbar(self.x, self.y, yerr=self.u, lw=lineWidth()*2, label=lbl)
        ax.fill_between(self.x, np.maximum(0, self.y-self.u), self.y+self.u,
                        color='red', lw=0, alpha=0.1, label="uncertainties")
        ax.legend(); ax.grid(); ax.set_xscale("log")

def plotResult(filename, withCountRate=False):
    filename = Path(filename)
    # CONTIN results
    cnt = Results(filename, rtype=ContinResult)
    if not hasattr(cnt, 'peakRanges'):
        return # nothing to do
    nsubplots = 2+len(cnt.peakRanges)+1
    fig, axes = plt.subplots(1, nsubplots, dpi=100, gridspec_kw=dict(wspace=.4))
    initFigure(fig, width=nsubplots*120, aspectRatio=nsubplots/1., quiet=True)
    fig.suptitle("…"+str(filename)[-60:], fontsize=10)
    cnt.plot(axes)
    plt.savefig(filename.with_suffix('.png'))
    return cnt

In [ ]:
results = [plotResult(resultDir, withCountRate=True) for resultDir in sorted(resultDirs)]